In [1]:
# %%
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, LSTM, Dense, Multiply, Lambda, Softmax, TimeDistributed
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# ----------------- Load & Preprocess -----------------
df = pd.read_csv("SQI.csv")
df = df.sort_values(['subject']).reset_index(drop=True)

df.columns = df.columns.str.lower()

# ----------------- Impute missing HR values -----------------
df['hr'] = df.groupby('subject')['hr'].transform(lambda x: x.fillna(x.mean()))
df['hr'] = df['hr'].fillna(df['hr'].mean())  # global mean fallback

features = ['bvp', 'temp', 'eda', 'hr']
target = 'sleep_stage'  # target

# Standardize features (fit only on training subjects later)
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

In [6]:
from scipy.stats import mode

# ----------------- Windowing -----------------
window_size = 60
step_size = 30
sequence_length = 5  # number of windows per sequence for inter-window modeling

def create_windows_per_subject(df, features, target, subjects_list):
    sequences = []
    labels = []
    for subject in subjects_list:
        df_sub = df[df['subject'] == subject].reset_index(drop=True)
        X_sub, y_sub = [], []
        # Create windows
        for i in range(0, len(df_sub) - window_size + 1, step_size):
            X_sub.append(df_sub[features].iloc[i:i+window_size].values)
            # Correct way to get mode in newer SciPy versions
            y_sub.append(mode(df_sub[target].iloc[i:i+window_size], keepdims=True).mode[0])
        X_sub = np.array(X_sub)
        y_sub = np.array(y_sub)
        # Create sequences of windows
        for i in range(0, len(X_sub) - sequence_length + 1, 1):
            sequences.append(X_sub[i:i+sequence_length])
            labels.append(mode(y_sub[i:i+sequence_length], keepdims=True).mode[0])
    return np.array(sequences), np.array(labels)

# ----------------- Split subjects -----------------
test_subjects = ['S015', 'S016']
train_subjects = [s for s in df['subject'].unique() if s not in test_subjects]

X_train, y_train = create_windows_per_subject(df, features, target, train_subjects)
X_test, y_test = create_windows_per_subject(df, features, target, test_subjects)

In [8]:
from tensorflow.keras.layers import Layer

# ----------------- Custom Attention Layer -----------------
class Attention(Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(1,),
                                 initializer='zeros',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, inputs):
        # inputs: (batch, timesteps, features)
        score = K.tanh(K.dot(inputs, self.W) + self.b)  # (batch, timesteps, 1)
        attention_weights = K.softmax(score, axis=1)     # (batch, timesteps, 1)
        context_vector = attention_weights * inputs     # (batch, timesteps, features)
        context_vector = K.sum(context_vector, axis=1)  # (batch, features)
        return context_vector

# ----------------- Hierarchical Model -----------------
num_windows = X_train.shape[1]
window_size = X_train.shape[2]
num_features = X_train.shape[3]

input_layer = Input(shape=(num_windows, window_size, num_features))

# Intra-window feature extraction using TimeDistributed
x = TimeDistributed(Conv1D(64, 3, activation='relu', padding='same'))(input_layer)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(Dropout(0.2))(x)
x = TimeDistributed(Conv1D(32, 3, activation='relu', padding='same'))(x)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(LSTM(64, return_sequences=True))(x)
x = TimeDistributed(Attention())(x)  # shape: (batch, num_windows, 64)

# Inter-window modeling
x = LSTM(64, return_sequences=True)(x)
x = TimeDistributed(Attention())(x)  # final context vector

# Output layer: multi-class classification
output = Dense(len(df[target].unique()), activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5, 60, 4)]        0         
                                                                 
 time_distributed_6 (TimeDi  (None, 5, 60, 64)         832       
 stributed)                                                      
                                                                 
 time_distributed_7 (TimeDi  (None, 5, 60, 64)         256       
 stributed)                                                      
                                                                 
 time_distributed_8 (TimeDi  (None, 5, 60, 64)         0         
 stributed)                                                      
                                                                 
 time_distributed_9 (TimeDi  (None, 5, 60, 32)         6176      
 stributed)                                                 

In [9]:
# ----------------- Train -----------------
# Use last 10% of training sequences as validation (sequentially)
val_split = int(len(X_train)*0.9)
X_train_final, X_val = X_train[:val_split], X_train[val_split:]
y_train_final, y_val = y_train[:val_split], y_train[val_split:]

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

history = model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50


773/773 [==============================] - 59s 70ms/step - loss: 1.4252 - accuracy: 0.4873 - val_loss: 1.4198 - val_accuracy: 0.5437
Epoch 2/50
773/773 [==============================] - 47s 61ms/step - loss: 1.2565 - accuracy: 0.5367 - val_loss: 1.3343 - val_accuracy: 0.5837
Epoch 3/50
773/773 [==============================] - 40s 52ms/step - loss: 1.1687 - accuracy: 0.5633 - val_loss: 1.4355 - val_accuracy: 0.4782
Epoch 4/50
773/773 [==============================] - 40s 51ms/step - loss: 1.0668 - accuracy: 0.6057 - val_loss: 1.3055 - val_accuracy: 0.5626
Epoch 5/50
773/773 [==============================] - 40s 52ms/step - loss: 0.9884 - accuracy: 0.6346 - val_loss: 1.4930 - val_accuracy: 0.4869
Epoch 6/50
773/773 [==============================] - 45s 58ms/step - loss: 0.9336 - accuracy: 0.6525 - val_loss: 1.4507 - val_accuracy: 0.4927
Epoch 7/50
773/773 [==============================] - 42s 55ms/step - loss: 0.8856 - accuracy: 0.6693 - val_loss: 1.5395 - val_accurac

In [10]:
# ----------------- Evaluate -----------------
y_pred = np.argmax(model.predict(X_test), axis=-1)
print("Accuracy:", accuracy_score(y_test, y_pred))


72/72 [==============================] - 3s 36ms/step
Accuracy: 0.39198606271777003


In [14]:
# %%
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Dropout, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from scipy.stats import mode

# ----------------- Load & Preprocess -----------------
df = pd.read_csv("SQI.csv")
df = df.sort_values(['subject']).reset_index(drop=True)

df.columns = df.columns.str.lower()

# Impute missing HR values
df['hr'] = df.groupby('subject')['hr'].transform(lambda x: x.fillna(x.mean()))
df['hr'] = df['hr'].fillna(df['hr'].mean())

features = ['bvp', 'temp', 'eda', 'hr', 'acc_x','acc_y','acc_z']
target = 'sleep_stage'

# ----------------- Normalize Per Subject -----------------
df_norm = pd.DataFrame()
for subj in df['subject'].unique():
    df_sub = df[df['subject'] == subj].copy()
    scaler = StandardScaler()
    df_sub[features] = scaler.fit_transform(df_sub[features])
    df_norm = pd.concat([df_norm, df_sub], axis=0)

df = df_norm.reset_index(drop=True)

In [15]:
# ----------------- Windowing -----------------
window_size = 60
step_size = 30

X_list, y_list, subj_list = [], [], []

for subject in df['subject'].unique():
    df_sub = df[df['subject'] == subject].reset_index(drop=True)
    for i in range(0, len(df_sub) - window_size + 1, step_size):
        X_list.append(df_sub[features].iloc[i:i+window_size].values)
        y_list.append(mode(df_sub[target].iloc[i:i+window_size], keepdims=True).mode[0])
        subj_list.append(subject)

X = np.array(X_list)
y = np.array(y_list)
subjects_per_window = np.array(subj_list)

In [16]:
# ----------------- Split by Subjects -----------------
test_subjects = ['S015', 'S016']
train_mask = ~np.isin(subjects_per_window, test_subjects)
test_mask = np.isin(subjects_per_window, test_subjects)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print(f"Train windows: {X_train.shape}, Test windows: {X_test.shape}")

Train windows: (13786, 60, 7), Test windows: (2304, 60, 7)


In [17]:
# ----------------- Simple 1D CNN + LSTM Model -----------------
input_layer = Input(shape=(window_size, len(features)))

x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = LSTM(64)(x)
output = Dense(len(np.unique(y)), activation='softmax')(x)  # multi-class classification

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 60, 7)]           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 60, 64)            1408      
                                                                 
 batch_normalization_6 (Bat  (None, 60, 64)            256       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 60, 64)            0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 6)                 390       
                                                           

In [18]:
# ----------------- Train -----------------
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50
776/776 [==============================] - 28s 31ms/step - loss: 1.1129 - accuracy: 0.5934 - val_loss: 1.3323 - val_accuracy: 0.5642
Epoch 2/50
776/776 [==============================] - 37s 48ms/step - loss: 0.9607 - accuracy: 0.6392 - val_loss: 1.4076 - val_accuracy: 0.5170
Epoch 3/50
776/776 [==============================] - 58s 75ms/step - loss: 0.8908 - accuracy: 0.6648 - val_loss: 1.5152 - val_accuracy: 0.5446
Epoch 4/50
776/776 [==============================] - 49s 63ms/step - loss: 0.8436 - accuracy: 0.6845 - val_loss: 1.5733 - val_accuracy: 0.4967
Epoch 5/50
776/776 [==============================] - 33s 43ms/step - loss: 0.8304 - accuracy: 0.6883 - val_loss: 1.4629 - val_accuracy: 0.5323
Epoch 6/50
776/776 [==============================] - 28s 37ms/step - loss: 0.8173 - accuracy: 0.6980 - val_loss: 1.6412 - val_accuracy: 0.4714
Epoch 7/50
776/776 [==============================] - 28s 36ms/step - loss: 0.7563 - accuracy: 0.7177 - val_loss: 1.6803 - val_accuracy:

In [19]:

# ----------------- Evaluate -----------------
y_pred = np.argmax(model.predict(X_test), axis=-1)
print("\nAccuracy:", accuracy_score(y_test, y_pred))

72/72 [==============================] - 2s 13ms/step

Accuracy: 0.40234375


In [21]:
X_train.shape

(13786, 60, 7)